<a href="https://colab.research.google.com/github/Rishitha744/Search-engines/blob/main/137009856_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset: Genius Lyrics Dataset

We are providing you with a small collection of the lyrics to 1000 songs collected from Genius (https://genius.com/). The full data was originally collected by Austin Benson at Cornell (https://www.cs.cornell.edu/~arb/data/genius-expertise/). The same as homework 1, you can use just the small set we provide: **lyrics_1000.jl**. You should treat each song as a unique document to be indexed by your system.

# Read and Parse the Lyrics Data (0 point)

Recall how we handled simple tokenization in Homework 1?
1. You should **lowercase** all words.
2. Remove song structure indicators.
(**strings in square brackets**, e.g., [Verse 1: Mike Shinoda & Chester Bennington])
3. Replace line breaks (e.g., \n, \n\n), punctuations, and special characters (\u2019, \u2005, etc.) with empty space (" ").
4. Tokenize the documents by splitting on whitespace.
5. Then only keep words that have a-zA-Z in them.

# Part 1: Text-based Approaches (50 points)

---



In this part, your job is to rank the documents rather than just provide set-based results as in Boolean Retrieval.

### A: Ranking with simple sums of TF-IDF scores (10 points)
For a multi-word query, we rank documents by a simple sum of the TF-IDF scores for the query terms in the document.
TF is the **log-weighted term frequency** $log_{10}(1+tf)$; and IDF is the log-weighted inverse document frequency $log_{10}(\frac{N}{df})$

**Output:**
You should output the top-5 results plus the TF-IDF sum score of each of these documents.

The output should be like this:

Rank Scores DocumentID Document

Now show the results for the query: `time`

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving lyrics_1000.jl to lyrics_1000 (1).jl


In [ ]:

import json
import re
import math
import pandas as pd
from collections import defaultdict, Counter

with open('lyrics_1000.jl', 'r', encoding='utf-8') as f:
    raw = [json.loads(line) for line in f]

def parse_lyrics(txt):
    txt = txt.encode('utf-8', 'ignore').decode('utf-8')
    txt = txt.replace('\\n', ' ')
    txt = txt.replace('\n', ' ')
    txt = re.sub(r'\[.*?\]', ' ', txt)
    txt = re.sub(r'[^a-zA-Z\s]', ' ', txt)
    txt = re.sub(r'\s+', ' ', txt).strip().lower()
    return txt

docs = {raw[i]['song']: parse_lyrics(raw[i].get('lyrics', '')) for i in range(len(raw))}
tokens = {song: docs[song].split() for song in docs}

N = len(tokens)
df = defaultdict(int)
for words in tokens.values():
    for w in set(words):
        df[w] += 1
idf = {w: math.log10(N / df[w]) for w in df}

tf = {}
for i, words in tokens.items():
    cnt = Counter(words)
    tf[i] = {t: math.log10(1 + cnt[t]) for t in cnt}

def rank_tfidf(query):
    query_terms = [w.lower() for w in re.findall(r'[a-zA-Z]+', query)]
    sc = defaultdict(float)
    for t in query_terms:
        if t not in idf:
            continue
        for doc_id in tf:
            if t in tf[doc_id]:
                sc[doc_id] += tf[doc_id][t] * idf[t]
    top5 = sorted(sc.items(), key=lambda x: x[1], reverse=True)[:5]
    original_lyrics = {r['song']: r.get('lyrics', '') for r in raw}

    res = pd.DataFrame([(i + 1, s, d, original_lyrics.get(d, '')[:100].replace('\n', ' ')) for i, (d, s) in enumerate(top5)], columns=['Rank', 'Score', 'DocumentID', 'Document'])
    return res

rank_tfidf("time")


,Rank,Score,DocumentID,Document
0,1,0.696832,Justin-bieber-one-time-lyrics,[Intro] Me plus you I'mma tell you one time ...
1,2,0.586521,Yelawolf-have-a-great-flight-lyrics,[Intro] Where's my calling angel? At the cof...
2,3,0.574922,The-beatles-christmas-time-is-here-again-lyrics,Christmas time is here again Christmas time ...
3,4,0.574922,Andy-mineo-friends-lyrics,[Verse 1] Hey now You never call me when you...
4,5,0.574922,John-legend-for-the-first-time-lyrics,[Produced by Dave Tozer & John Legend] [Ver...


\Now show the results for the query: `never know`

In [ ]:
rank_tfidf("never know")

,Rank,Score,DocumentID,Document
0,1,1.382408,Jhene-aiko-lyin-king-lyrics,"[Verse 1] Okay, so you just Go around breaki..."
1,2,1.164265,Grimes-genesis-lyrics,"[Chorus] My heart, I never be, I never see, ..."
2,3,0.961624,The-1975-chocolate-lyrics,[Verse 1] Call it a split cause you know tha...
3,4,0.851125,Chris-brown-another-you-lyrics,[Intro] K-K-K-K-K-Mac [Verse 1: Chris Brown...
4,5,0.848041,One-direction-live-while-were-young-lyrics,"[Intro] [Verse 1: Liam & Zayn] Hey girl, I'..."


Now show the results for the query: `make sense`

In [ ]:
rank_tfidf("make sense")

,Rank,Score,DocumentID,Document
0,1,1.233371,Tame-impala-music-to-walk-home-by-lyrics,[Verse] But that's only while I think of you...
1,2,1.204120,Florence-the-machine-all-this-and-heaven-too-l...,[Verse 1] And the heart is hard to translate...
2,3,0.954281,Genius-how-to-annotate-and-edit-on-genius-anno...,✧ A Genius annotation is a note that explain...
3,4,0.954243,Ellie-goulding-paradise-lyrics,[Verse 1] I've been filling in the blanks Yo...
4,5,0.881189,Gorillaz-kids-with-guns-lyrics,[Verse 1] Kids with guns Kids with guns Taki...


### B: Ranking with vector space model with TF-IDF (15 points)

**Cosine:** You should use cosine as your scoring function.

**TFIDF:** For the **document vectors**, use the standard TF-IDF scores as **introduced in A**. For the **query vector**, use **simple weights (the raw term frequency)**. For example:
* query: never $\rightarrow$ (1)
* query: never know $\rightarrow$ (1, 1)

**Output:**
You should output the top-5 results plus the cosine score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

---

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!

Now show the results for the query: `time`

In [ ]:
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
import re
import math

doc_vecs = {}
for song_id, term_freqs in tf.items():
    doc_vecs[song_id] = {}
    norm = 0.0
    for term, tf_val in term_freqs.items():
        if term in idf:
            weight = tf_val * idf[term]
            doc_vecs[song_id][term] = weight
            norm += weight ** 2
    norm = math.sqrt(norm)
    if norm > 0:
        for term in doc_vecs[song_id]:
            doc_vecs[song_id][term] /= norm

def rank_tfidf_cosine(query):
    q_terms = [t.lower() for t in re.findall(r'[a-zA-Z]+', query)][:3]
    q_counts = Counter(q_terms)
    q_norm = math.sqrt(sum(v ** 2 for v in q_counts.values()))
    if q_norm == 0:
        return pd.DataFrame(columns=['Rank', 'Score', 'DocumentID', 'Document'])
    q_vec = {t: v / q_norm for t, v in q_counts.items()}

    scores = defaultdict(float)
    for song_id, dvec in doc_vecs.items():
        score = 0.0
        for term, qv in q_vec.items():
            if term in dvec:
                score += qv * dvec[term]
        scores[song_id] = score

    top5 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]
    original_lyrics = {r['song']: r.get('lyrics', '') for r in raw}
    res = pd.DataFrame([(i+1, round(s, 4), d, original_lyrics.get(d, '')[:120].replace('\n', ' '))for i, (d, s) in enumerate(top5)],
        columns=['Rank', 'Score', 'DocumentID', 'Document'])
    return res

rank_tfidf_cosine("time")


,Rank,Score,DocumentID,Document
0,1,0.1648,The-beatles-any-time-at-all-lyrics,"[Chorus] Any time at all, any time at all An..."
1,2,0.1513,John-legend-for-the-first-time-lyrics,[Produced by Dave Tozer & John Legend] [Ver...
2,3,0.1321,Zacari-redemption-interlude-lyrics,[Verse: Zacari] You love it when I fall on m...
3,4,0.1298,Xxxtentacion-what-are-you-so-afraid-of-lyrics,[Chorus] What are you so afraid of? Is it lo...
4,5,0.1187,The-neighbourhood-you-get-me-so-high-lyrics,[Verse 1] Hope you don't regret it I pushed ...


Now show the results for the query: `never know`

In [ ]:
# your code here
rank_tfidf_cosine("never know")

,Rank,Score,DocumentID,Document
0,1,0.2045,Jhene-aiko-lyin-king-lyrics,"[Verse 1] Okay, so you just Go around breaki..."
1,2,0.1819,Tame-impala-beverly-laurel-lyrics,Why do they say that they know what's right ...
2,3,0.1724,Grimes-genesis-lyrics,"[Chorus] My heart, I never be, I never see, ..."
3,4,0.1381,Rm-uhgood-lyrics,RM 어긋 가사 [Refrain] All I need is me All I n...
4,5,0.1255,Joji-you-suck-charlie-lyrics,"[Intro] ま、今夜は本気でやってもらいますね。 One, two, three, ..."


Now show the results for the query: `make sense`

In [ ]:
# your code here
rank_tfidf_cosine("make sense")

,Rank,Score,DocumentID,Document
0,1,0.2030,Tame-impala-music-to-walk-home-by-lyrics,[Verse] But that's only while I think of you...
1,2,0.1565,Chris-brown-make-love-lyrics,"[Verse 1] I wanna make love, yeah I wanna ma..."
2,3,0.1355,Tame-impala-why-wont-you-make-up-your-mind-lyrics,[Chorus] Why won't you make up your mind? Gi...
3,4,0.1214,Florence-the-machine-all-this-and-heaven-too-l...,[Verse 1] And the heart is hard to translate...
4,5,0.1066,Ellie-goulding-paradise-lyrics,[Verse 1] I've been filling in the blanks Yo...


### C: Ranking with BM25 (15 points)
Finally, let's try the BM25 approach for ranking. You could choose $k_1 = 1.2$ and $b = 0.75$ but feel free to try other options.

**Output:**
You should output the top-5 results plus the BM25 score of each of these documents.  

The output should be like this:

Rank Scores DocumentID Document

Now show the results for the query: `time`

In [ ]:
import math
import re
import pandas as pd
from collections import defaultdict, Counter

k1 = 1.2
b = 0.75

doc_len = {doc: len(words) for doc, words in tokens.items()}
avg_len = sum(doc_len.values()) / len(doc_len)

def rank_bm25(query):
    query_terms = [t.lower() for t in re.findall(r'[a-zA-Z]+', query)][:3]
    q_count = Counter(query_terms)
    scores = defaultdict(float)

    for term in q_count:
        if term not in idf:
            continue
        idf_term = idf[term]

        for doc_id, words in tokens.items():
            f = words.count(term)
            if f == 0:
                continue
            dl = doc_len[doc_id]
            denom = f + k1 * (1 - b + b * dl / avg_len)
            score_term = idf_term * ((f * (k1 + 1)) / denom)
            scores[doc_id] += score_term

    top5 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]
    original_lyrics = {r['song']: r.get('lyrics', '') for r in raw}
    result = pd.DataFrame([(i + 1, round(s, 4), d, original_lyrics.get(d, '')[:120].replace('\n', ' ')) for i, (d, s) in enumerate(top5)],
        columns=["Rank", "Score", "DocumentID", "Document"]
    )
    return result

rank_bm25("time")


,Rank,Score,DocumentID,Document
0,1,0.9646,Justin-bieber-one-time-lyrics,[Intro] Me plus you I'mma tell you one time ...
1,2,0.9617,The-beatles-christmas-time-is-here-again-lyrics,Christmas time is here again Christmas time ...
2,3,0.9585,The-beatles-any-time-at-all-lyrics,"[Chorus] Any time at all, any time at all An..."
3,4,0.9567,John-legend-for-the-first-time-lyrics,[Produced by Dave Tozer & John Legend] [Ver...
4,5,0.9567,Zacari-redemption-interlude-lyrics,[Verse: Zacari] You love it when I fall on m...


Now show the results for the query: `never know`

In [ ]:
# your code here
rank_bm25("never know")

,Rank,Score,DocumentID,Document
0,1,1.7374,Jhene-aiko-lyin-king-lyrics,"[Verse 1] Okay, so you just Go around breaki..."
1,2,1.6968,Grimes-genesis-lyrics,"[Chorus] My heart, I never be, I never see, ..."
2,3,1.5739,Gorillaz-sound-check-gravity-lyrics,[Intro: 2D] Gravity-ay-ay-ay-ay-ay on me Nev...
3,4,1.5717,The-1975-chocolate-lyrics,[Verse 1] Call it a split cause you know tha...
4,5,1.5523,The-beatles-oh-darling-lyrics,"[Verse 1] Oh! Darling, please believe me I'l..."


Now show the results for the query: `make sense`

In [ ]:
# your code here
rank_bm25("make sense")

,Rank,Score,DocumentID,Document
0,1,3.6349,Tame-impala-music-to-walk-home-by-lyrics,[Verse] But that's only while I think of you...
1,2,3.0299,Gorillaz-kids-with-guns-lyrics,[Verse 1] Kids with guns Kids with guns Taki...
2,3,2.9675,Florence-the-machine-all-this-and-heaven-too-l...,[Verse 1] And the heart is hard to translate...
3,4,2.5423,Ellie-goulding-paradise-lyrics,[Verse 1] I've been filling in the blanks Yo...
4,5,2.3053,Linkin-park-no-roads-left-lyrics,[Verse 1: Mike Shinoda] Standing alone with ...


### Discussion (10 points)
Briefly discuss the differences you see between the three methods. You should try additional queries beyond the ones we list. Is there a ranking approach you prefer? Explain and give concrete examples.


1)TF IDF ranks documents by summing the frequency of weighted terms but it lacks normalization and doesn't consider document length there by leading to document length bias. Which is why in TF IDF Justin bieber's One time document ranked higher for the query term 'time' due to the high frequency of word "time in this document. To summarize, if a document has a more matching frequency then this model scores that document has highest without considering length normalization and semantics

2)But Vector space model normalize the document length and there by reducing document length bias. Which is why 'The beattles any time at all" document ranked highest as this model calculates how semantically this document is appropriate instead of just giving the document which has more frequency of the query term

3)BM25 balances the ranking using saturization k1 and normalization b terms. It balances both frequency and length and there by providing the better output than the above 2 models. The result is more realistic and is similar to the human ranking. Which is why it ranked 'Justin Bieber' and 'the beattles' highly for the query 'time' there by showing it considers and balances both relevance and term frequency properly.

I would prefer BM25 algorithm as it ranks the document based on both document length and raw frequency. It normalizes long documents and also saturates term frequency which means after a saturation point even though the term is being repeated it adds little weightage to the ranking score which is true in the real world unlike TF IDF where it consider raw frequency and rank accordingly.BM25 ranking is closest to the human ranking therefore this model is more preferable.

# Part 2: Link-based Approaches (50 points)




In this part, we're going to adapt the classic PageRank and  Hubs-and-Authorities approaches to allow us to find the most important artists. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we view it as artists in a network.

Over this graph, we can apply classic methods like PageRank and Hubs-and-Authorities to order the artists according to some notion of importance.

First, you will run following code to construct the graph based on the given file **song_info.json**. In this graph, the node is the artist and a link between two artist is added if their songs share any genre tags. For example, the genre tags shared by artist `Kendrick-lamar` and `A-ap-rocky` are 'Non-Music', 'Rap', 'Demo', ... Thus, there is an edge between node `Kendrick-lamar` and `A-ap-rocky`.

You need to replace the file path for the following code:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
from google.colab import files

uploaded = files.upload()

with open('song_info.json', 'r', encoding='utf-8') as f:
     data = [json.loads(line) for line in f]

Saving song_info.json to song_info (2).json


Then, run the following code to construct the graph.

In [ ]:
songs = data[0:50000]
sample=so[0:2]
print(sample)
# Initialize a dictionary to map artists to a set of tags
artist_tags_map = {}

# Populate the artist_tags_map with tags associated with each artist
for song in songs:
    artist = song['primary_artist']
    tags = set(song['tags'])
    if artist not in artist_tags_map:
        artist_tags_map[artist] = tags
    else:
        artist_tags_map[artist] = artist_tags_map[artist].union(tags)

NameError: name 'data' is not defined

In [ ]:
# Create a graph
G = nx.Graph()

# Add nodes for each artist
for artist in artist_tags_map:
    G.add_node(artist)

# Add edges based on shared tags using the artist_tags_map
artists = list(artist_tags_map.keys())  # List of artists for iteration
for i in tqdm(range(len(artists)), desc="Processing artists"):
    for j in range(i + 1, len(artists)):
        shared_tags = artist_tags_map[artists[i]].intersection(artist_tags_map[artists[j]])
        if shared_tags:
            # Add an edge between artists if they share any tag
            G.add_edge(artists[i], artists[j], weight=len(shared_tags))

Processing artists: 100%|██████████| 6974/6974 [01:16<00:00, 91.66it/s] 


### A: PageRank (10 points)
Given this graph, your first job is to simply run PageRank to find the highest ranking artists. You may use the [built-in pagerank](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html) function in networkx.

Print the top-5 highest PageRank scores in `artist: score` format.

In [ ]:
# your code here

import networkx as nx

scores = nx.pagerank(G)
top_artists = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]

print("Top 5 artists by PageRank:\n")
for artist, value in top_artists:
    print(f"{artist}: {value:.6f}")


Top 5 artists by PageRank:

Kanye-west: 0.000429
Xxxtentacion: 0.000402
Beyonce: 0.000395
Taylor-swift: 0.000390
Rihanna: 0.000389


### B: Personalized PageRank (10 points)
For this problem assume the artist of interest is `"Taylor-swift"`. In other words, the seed set has only one node: `"Taylor-swift"`. You should compute the personalized PageRank and print the top-5 highest personalized PageRank scores in `artist: score` format. Note that there is a nice function parameter called "personalization" in the built-in pagerank that you can use for this problem.

In [ ]:
# your code here

import networkx as nx

focus_artist = "Taylor-swift"
p = {node: 0 for node in G.nodes()}
if focus_artist in p:
    p[focus_artist] = 1

personal_ranks = nx.pagerank(G, personalization=p)
top_five = sorted(personal_ranks.items(), key=lambda x: x[1], reverse=True)[:5]

print("Top 5 artists by Personalized PageRank (Taylor-swift):\n")
for artist, score in top_five:
    print(f"{artist}: {score:.6f}")


Top 5 artists by Personalized PageRank (Taylor-swift):

Taylor-swift: 0.150357
Kanye-west: 0.000765
Rihanna: 0.000714
Ariana-grande: 0.000696
Lana-del-rey: 0.000681


### C: Hubs and Authorities (20 points)
Just like we can calculate PageRank over the artist graph, we can also find the hub and authority scores for artists.

For this part, you should return the top-5 artists with highest Hub scores and the top-5 artists with the highest Authority scores. The **output** should be like:

`Hub Scores`

`1. user1: score1`

`...`

`Authority Scores`

`1. user1: score1`

` ...`




In [ ]:
# Your code here
import networkx as nx

hub_vals, auth_vals = nx.hits(G, max_iter=100)

top_hubs = sorted(hub_vals.items(), key=lambda x: x[1], reverse=True)[:5]
top_auth = sorted(auth_vals.items(), key=lambda x: x[1], reverse=True)[:5]

print("Hub Scores\n")
for i, (artist, val) in enumerate(top_hubs, start=1):
    print(f"{i}. {artist}: {val:.6f}")

print("\nAuthority Scores\n")
for i, (artist, val) in enumerate(top_auth, start=1):
    print(f"{i}. {artist}: {val:.6f}")


Hub Scores

1. Kanye-west: 0.000467
2. Xxxtentacion: 0.000419
3. Drake: 0.000416
4. Eminem: 0.000413
5. Kendrick-lamar: 0.000411

Authority Scores

1. Kanye-west: 0.000467
2. Xxxtentacion: 0.000419
3. Drake: 0.000416
4. Eminem: 0.000413
5. Kendrick-lamar: 0.000411


### Discussion (10 points)
Briefly discuss the differences you see between the three methods. You should play with the personalization parameters in PageRank and show some findings.


1) Page Rank measures overall global importance based on their overall connectivity points in the graph which is why Taylor Swift and Kanye West rated higher
2) In Personalized Page rank, we add a bias and focus on one node, in this case Taylor Swift and results are spread across artists who are connected to Taylor Swift in genre or other factors. Hence we are seeing Ariana Grande and Lana-del-rey in the results as these artists genre are nearly connected to the Taylor swift's genre
3) Authorities are artists that are highly referenced and hubs are artists or pages that link to good authorities. Hubs acts are recommenders. Kanye west ranks higher here denoting he has been widely connected and referenced as well. But Taylor swift doesn't appear in the results as this artist doesn't have strong mutual influence*your discussion here*